In [3]:
import pandas as pd
import numpy as np



In [12]:
df= pd.read_csv('/content/SIMPrice.csv')
df

,sim_number,price_vnd
0,343189288,450000
1,888899580,3000000
2,928960006,500000
3,902438679,5000000
4,334307889,450000
...,...,...
199995,866161769,450000
199996,708124126,1000000
199997,904755200,1000000
199998,329220204,450000


In [13]:
df.describe()

,sim_number,price_vnd
count,2.000000e+05,2.000000e+05
mean,7.304797e+08,1.395027e+07
std,2.406479e+08,5.912997e+08
min,3.250098e+08,9.900000e+04
25%,3.922605e+08,5.000000e+05
50%,8.334287e+08,1.000000e+06
75%,9.180874e+08,5.000000e+06
max,9.979797e+08,1.680000e+11


In [14]:
df['price_vnd'].value_counts().head(100)

,count
price_vnd,
450000,45119
1000000,38472
500000,30619
3000000,29199
5000000,21383
...,...
12200000,40
13400000,40
11200000,39


data phân bố không đồng đều, vậy nên ta sẽ đặt ra bài toán dự đoán theo từng khoảng giá sao cho fit với tệp data. Ở đây tôi có 1 file 200k data với giá tiền của sim từ 450k ->
So we split the data to different ranges

In [15]:
def sim_category(x):
  if x['price_vnd']<=450000:
    return 0
  elif x['price_vnd']<=500000:
    return 1
  elif x['price_vnd']<=1000000:
    return 2
  elif x['price_vnd']<=3000000:
    return 3
  elif x['price_vnd']<=5000000:
    return 4
  else:
    return 5


In [16]:
df['sim_price_cate']=df.apply(sim_category, axis=1)
df

,sim_number,price_vnd,sim_price_cate
0,343189288,450000,0
1,888899580,3000000,3
2,928960006,500000,1
3,902438679,5000000,4
4,334307889,450000,0
...,...,...,...
199995,866161769,450000,0
199996,708124126,1000000,2
199997,904755200,1000000,2
199998,329220204,450000,0


In [17]:
df['sim_price_cate'].value_counts()

,count
sim_price_cate,
0,45543
2,38472
5,34784
1,30619
3,29199
4,21383


In [18]:

X=[]
y=[]
for index,row in df.iterrows():
  X.append([int(c) for c in str(row['sim_number'])])
  y.append(row['sim_price_cate'])
X

[[3, 4, 3, 1, 8, 9, 2, 8, 8],
 [8, 8, 8, 8, 9, 9, 5, 8, 0],
 [9, 2, 8, 9, 6, 0, 0, 0, 6],
 [9, 0, 2, 4, 3, 8, 6, 7, 9],
 [3, 3, 4, 3, 0, 7, 8, 8, 9],
 [3, 2, 8, 1, 9, 0, 6, 8, 0],
 [9, 2, 6, 0, 5, 2, 0, 0, 5],
 [9, 2, 1, 2, 2, 0, 9, 2, 4],
 [8, 7, 7, 0, 0, 1, 8, 6, 6],
 [7, 6, 9, 8, 8, 9, 9, 9, 9],
 [8, 7, 9, 1, 6, 1, 7, 8, 6],
 [3, 4, 3, 5, 5, 8, 7, 9, 9],
 [8, 6, 6, 6, 1, 5, 6, 7, 9],
 [8, 6, 9, 8, 5, 6, 8, 9, 2],
 [3, 4, 3, 6, 8, 4, 3, 8, 6],
 [8, 2, 7, 8, 0, 8, 0, 8, 8],
 [7, 8, 6, 0, 0, 8, 8, 8, 8],
 [7, 6, 3, 6, 5, 9, 2, 5, 9],
 [9, 7, 1, 3, 1, 8, 5, 6, 8],
 [3, 7, 7, 3, 2, 1, 4, 8, 6],
 [8, 6, 2, 9, 9, 8, 9, 3, 7],
 [8, 6, 5, 4, 8, 7, 0, 7, 9],
 [5, 6, 6, 6, 8, 2, 8, 6, 8],
 [8, 3, 4, 1, 4, 1, 1, 4, 1],
 [9, 1, 6, 7, 8, 9, 8, 1, 5],
 [8, 5, 6, 6, 3, 7, 6, 7, 8],
 [8, 4, 7, 7, 9, 7, 9, 8, 8],
 [9, 0, 4, 4, 3, 4, 8, 8, 0],
 [9, 7, 9, 7, 9, 2, 7, 4, 3],
 [7, 6, 5, 9, 5, 9, 6, 9, 5],
 [8, 2, 5, 2, 9, 4, 6, 7, 8],
 [3, 8, 4, 0, 7, 6, 8, 8, 8],
 [9, 0, 1, 1, 8, 4, 6, 7, 6],
 [3, 3, 4,

In [19]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y) # <--- truyền y vào đây
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_one_hot = onehot_encoder.fit_transform(integer_encoded)
print(y_one_hot)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.1, random_state=42)
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam



model = Sequential()
# 3 lớp LSTM chồng nhau có trả về sequence
model.add ( LSTM(units=512, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add (Dropout(0.2))
model.add ( LSTM(units=512, return_sequences=True))
model.add (Dropout(0.2))
model.add ( LSTM(units=512, return_sequences=True))
model.add (Dropout(0.2))
# 1 lớp LSTM không trả về sequence
model.add ( LSTM(units=128, return_sequences=False))
model.add (Dropout(0.2))
# Đưa qua 1 lớp Dense
model.add ( Dense(units=512))
# Lớp output ra kết quả
model.add ( Dense(units=6, activation = "softmax"))

optimizer = Adam()
model.compile(loss = "categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 9, 512)              │       1,052,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 9, 512)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 9, 512)              │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 9, 512)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 9, 512)              │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 9, 512)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │         328,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,648,390 (21.55 MB)

 Trainable params: 5,648,390 (21.55 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint_callback = ModelCheckpoint(
    filepath="best_model.weights.h5",
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=True, verbose=1)

history = model.fit(X_train, y_train, epochs = 30, batch_size = 64, validation_data = (X_test, y_test), callbacks = [model_checkpoint_callback])
# Val Accuracy : 0.78


model.load_weights("/best_model.weights.h5")

Epoch 1/30
2810/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4046 - loss: 1.4863
Epoch 1: val_accuracy improved from -inf to 0.60895, saving model to best_model.weights.h5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 54s 17ms/step - accuracy: 0.4047 - loss: 1.4861 - val_accuracy: 0.6090 - val_loss: 1.0261
Epoch 2/30
2810/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6137 - loss: 1.0120
Epoch 2: val_accuracy improved from 0.60895 to 0.67760, saving model to best_model.weights.h5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 79s 17ms/step - accuracy: 0.6137 - loss: 1.0119 - val_accuracy: 0.6776 - val_loss: 0.8367
Epoch 3/30
2811/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6734 - loss: 0.8515
Epoch 3: val_accuracy improved from 0.67760 to 0.71750, saving model to best_model.weights.h5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 48s 17ms/step - accuracy: 0.6734 - loss: 0.8515 - val_accuracy: 0.7175 - val_loss: 0.7410
Epoch 4/30
2812/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7114 - loss: 0.7520
Ep

KeyboardInterrupt: 

In [24]:
import random

for i in range(0,5):
  idx = random.randint(0,20000)
  result = model.predict(np.expand_dims(X_test[idx], axis=0))
  print("Số sim = ", X_test[idx])
  print("Kết quả dự đoán = ", result)
  print("Index dự đoán = ", np.argmax(result))
  print("Giá trị thật = ", y_test[idx])
  print("Đúng/sai = ", np.argmax(result) == np.argmax(y_test[idx]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
Số sim =  [8 2 6 0 4 3 6 7 8]
Kết quả dự đoán =  [[4.5002253e-08 1.4080805e-05 8.2217099e-04 4.5980636e-05 1.7407150e-05
  9.9910033e-01]]
Index dự đoán =  5
Giá trị thật =  [0. 0. 0. 0. 0. 1.]
Đúng/sai =  True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Số sim =  [9 6 2 2 9 0 1 9 0]
Kết quả dự đoán =  [[5.9852078e-05 6.6884146e-03 5.7543875e-03 3.5009336e-01 3.2824463e-01
  3.0915931e-01]]
Index dự đoán =  3
Giá trị thật =  [0. 0. 0. 1. 0. 0.]
Đúng/sai =  True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Số sim =  [9 8 9 0 4 7 0 5 0]
Kết quả dự đoán =  [[0.10528669 0.59096783 0.28014767 0.0157035  0.00558457 0.00230979]]
Index dự đoán =  1
Giá trị thật =  [1. 0. 0. 0. 0. 0.]
Đúng/sai =  False
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Số sim =  [7 7 8 9 4 5 6 7 8]
Kết quả dự đoán =  [[1.6593390e-08 4.9079375e-08 3.5517517e-06 1.0204264e-03 1.1904670e-03
  9.9778545e-01]]
Index dự đoán =  5
Giá trị thật =  [0. 0. 0. 0. 0. 1.]
Đúng/sai =  True
1/1 ━━━━━━━━━━━━━━━━━━